# Analysis

### Importing stuff

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
import sys 

sys.path.append(os.path.join(os.getcwd(), '..'))
from facemap_tools.process_tools import create_procs
import facemap_tools.plot_utils
from facemap_tools.plotting import plot_area
import facemap

In [ ]:
### Problems with loading .npy-files in the gui:
proc_broken = np.load("/Users/annapaulinehjertvikaasen/Documents/2. UiO/Sommerjobb - Frederik/Sommerjobb 2023/MouseProject/ING71_MEC_230309/AVI/ING71_MEC_230309_000_Behav_Fr1-9973_proc.npy", allow_pickle=True).item()
print(proc_broken)
print(proc_broken['rois']) ## seems like it is not broken 


In [ ]:
## Now 230310 works (with its own )
proc_230310 = np.load("/Users/annapaulinehjertvikaasen/Documents/2. UiO/Sommerjobb - Frederik/Sommerjobb 2023/MouseProject/ING71_MEC_230310/AVI/ING71_MEC_230310_000_Behav_Fr1-9973_proc.npy", allow_pickle=True).item()
print(proc_230310)

### Loading procs

In [ ]:
# resultfolder = "/Volumes/T7/ING71_MEC_230309/AVI/resultsnpy" 
resultfolder = "/Users/annapaulinehjertvikaasen/Documents/2. UiO/Sommerjobb - Frederik/Sommerjobb 2023/MouseProject/ING71_MEC_230309/AVI/resultsnpy"

procs = create_procs(resultfolder)

### Looking into pupil area plotted 

In [ ]:
for filename in list(procs.keys())[::5]:
    proc = procs[filename]
    plot_area(proc)
    plot_area(proc, zoom=True)


In [ ]:
proc = list(procs.values())[-1]

plt.plot(proc['pupil'][0]['area'])
plt.plot(proc['pupil'][0]['area_smooth'])
plt.xlim((500,800))

In [ ]:
print(procs.keys())

In [ ]:
plt.plot(procs['ING71_MEC_230309_001_Behav_Fr1-11947_proc']['pupil'][0]['area'])
plt.plot(procs['ING71_MEC_230309_001_Behav_Fr1-11947_proc']['pupil'][0]['area_smooth'])
# plt.xlim((500,800))
# plt.ylim((100,125))

### More interesting data?

In [ ]:
proc = list(procs.values())[0]

print(proc.keys())
print(proc['pupil'][0].keys())

print(np.shape(proc['pupil'][0]['axdir']))
print(np.shape(proc['pupil'][0]['axlen']))
# print(proc['pupil'][0]['axdir'])
# print(proc['pupil'][0]['axlen'])

In [ ]:
plt.plot(proc['pupil'][0]['axlen'][:,0])
plt.plot(proc['pupil'][0]['axlen'][:,1])

Looks like axlen and axdir refer to the axis that make up the tracker ellipse around the pupil. They seem to coincide very well with the area, which is expected as the area is calculated from the ellipse. We assume that axdir is the direction of the two axis making up the ellipse, and axlen the length of them. 

In [ ]:
print(proc['rois'][0].keys())

Looks like rois contain little data of interest for further analysis. It could be interesting for understanding what we have done when creating the ROIs, so one could argue that the proc-files (per folder, as the ROI is the same for an entire folder. Could call it sample or something) should be saved. 

In [ ]:
# 'Ly', 'Lx' 
print(proc['Lx'])
print(proc['Ly'])
print(86/3*66/3)

We assume the pupil makes up about 1/3 of the x-dir and y-dir. We see that the number of pixels within this fram seem to correspond with the y-axis of the area plots. 

When trying with two different files (within the same folder). Lx and Ly stay constant

In [ ]:
# 'avgframe':  list of average frames for each video from a subset of frames (binned by sbin) 
# 'avgmotion':  list of average motions for each video from a subset of frames (binned by sbin)
print(np.shape(proc['avgframe']))
print(proc['avgframe'])
plt.plot(proc['avgframe'][0])

In [ ]:
print(np.shape(proc['avgmotion']))
print(proc['avgmotion'])
plt.plot(proc['avgmotion'][0])

### Downsampling 

In [ ]:
import scipy as sp

In [ ]:
#Behavioral data

def get_experiment_n_day_k(n, k):
    #n is a string ex: '04', '10'
    #k is on the form yymmdd
    base = "/Volumes/T7/ING71_MEC_" + str(k)
    base = os.path.join(base, 'AVI')
    base = os.path.join(base, 'resultsnpy')

    behavior_base = "/Volumes/T7/mec-lec-por-data"
    files = os.listdir(base)

    experiment_base = 'ING71_MEC_' + str(k) + '_0' + n

    exp_filenames = [file for file in files if file[:20] == experiment_base]

    complete_proc = []
    for file in exp_filenames:
        proc = np.load(os.path.join(base, file), allow_pickle=True).item()
        complete_proc.extend(proc['pupil'][0]['area'])

    negative = np.load(os.path.join(behavior_base, experiment_base + '_negative.npy'), allow_pickle=True)
    positive = np.load(os.path.join(behavior_base, experiment_base + '_positive.npy'), allow_pickle=True)
    reward = np.load(os.path.join(behavior_base, experiment_base + '_reward.npy'), allow_pickle=True)
    aversive = np.load(os.path.join(behavior_base, experiment_base + '_aversive.npy'), allow_pickle=True)
    
    return complete_proc, negative, positive, reward, aversive

In [ ]:
c_proc, negative ,positive, reward, aversive = get_experiment_n_day_k('03', 230309)

In [ ]:
n_points = len(positive)
ds_proc = sp.signal.resample(c_proc, n_points) #ds stands for downsampled

plt.plot(ds_proc)
plt.plot(c_proc)
plt.show()

#Plot with downsampeled 
plt.plot(ds_proc)
plt.scatter(np.arange(len(positive)), positive*ds_proc, 5, color='r')
# plt.xlim(0,100)

In [ ]:

# Using decimate

ds_proc_d  = sp.signal.decimate(c_proc, 2)
 
plt.plot(ds_proc_d)
plt.plot(c_proc)
plt.show()

## Checking various ROIs


